In [1]:
# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import csv
import glob
import os
import cassandra
from typing import List
import sql_queries

#### Creating list of filepaths to process original event csv data files

In [3]:
from cassandra.cluster import Cluster, Session

cluster = Cluster(['127.0.0.1'])
cassandra_session = cluster.connect()

# create and use schema
cassandra_session.execute("""
            CREATE KEYSPACE IF NOT EXISTS event_data
            WITH REPLICATION =
            { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

cassandra_session.set_keyspace('event_data')

print("connected to cassandra")

connected to cassandra


In [4]:
class CsvRecord:
    """
    utility class to make transform from csv to columns clearer
    """
    good_record: bool = False  # always false unless all fields were found

    def __init__(self, line : List[any]):
        """
        transform one values array into named columns    
        :param 
            line: array of 11 values
        """
        if len(line) != 11:
            print(f"bad line: {','.join(line)}")
            return
        self.artist = line[0]
        self.first_name= line[1]
        self.gender = line[2]
        self.item_in_session = int(line[3])
        self.last_name= line[4]
        self.song_length = float(line[5])
        self.level = line[6]
        self.location= line[7]
        self.session_id= int(line[8])
        self.song = line[9]
        self.user_id = int(line[10])
        self.good_record = True
                  
print("class created")

class created


In [5]:
def merge_csv_data(file_path_list: List[str]) -> str:
    """
    merge a list of csv files into a single file 
    
    :param 
        file_path_list: a list of csv path names
    
    :return
        the merged single fle name (hard coded to event_datafile_new.csv)
    """

    # initiating an empty list of rows that will be generated from each file
    destination_file = 'event_datafile_new.csv'
    full_data_rows_list = []

    # for every filepath in the file path list
    for f in file_path_list:

        # reading csv file
        with open(f, 'r', encoding='utf8', newline='') as csvfile:
            # creating a csv reader object
            csvreader = csv.reader(csvfile)
            next(csvreader)

            # extracting each data row one by one and append it
            for line in csvreader:
                full_data_rows_list.append(line)

            # uncomment the code below if you would like to get total number of rows

    # print(len(full_data_rows_list))
    # uncomment the code below if you would like to check to see what the list of event data rows will look like
    # print(full_data_rows_list)

    # creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
    # Apache Cassandra tables
    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    dumped_count = 0
    hits_count = 0

    with open(destination_file, 'w', encoding='utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
                         'level', 'location', 'sessionId', 'song', 'userId'])
        for row in full_data_rows_list:
            if row[0] == '':
                dumped_count += 1
                continue
            hits_count += 1
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

    print(f"dumped {dumped_count} records")
    print(f"kept {hits_count} records")

    # check the number of rows in your csv file
    with open(destination_file, 'r', encoding='utf8') as f:
        print(f"wrote {sum(1 for line in f)} lines to event_datafile_new.csv")

    return destination_file

print("method merge_csv_data created")

method merge_csv_data created


In [6]:
def get_files(event_data: str) -> List[str]:
    """
    walk root directory and get all csv files in all subdirectories  
    
    :param 
        event_data: the root directory name to walk
    
    :return
        A list of all csv files in all subdirectories
    """

    # checking your current working directory
    print(f"Getting csv files from {os.getcwd()}")

    # Get your current folder and subfolder event data
    filepath = os.getcwd() + '/' + event_data

    # Create a for loop to create a list of files and collect each filepath
    for root, dirs, files in os.walk(filepath):
        # join the file path and roots with the subdirectories using glob
        file_path_list = glob.glob(os.path.join(root, '*.csv'))

    return file_path_list

print("method get_files created")

method get_files created


In [7]:
def load_csv_data(file: str, session: Session):
    """
    read the master csv file and load into cassadra tables  
    
    :param 
        file: the master csv file
        session: a cassandra.cluster.Session    
    """

    print("loading database...")
    line_count = 0

    # preconditions
    if not os.path.exists(file):
        raise (Exception(f"file named '{file}' was not found!!!"))
    if session is None:
        raise (Exception("Cassandra session is not set!!!!"))

    with open(file, encoding='utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)  # skip header
        for line in csvreader:
            line_count += 1

            # read line array into a class instance
            r = CsvRecord(line)

            # song play sessions
            session.execute(sql_queries.insert_songplay_sessions, (
                r.session_id, r.item_in_session, r.artist, r.song, r.song_length
            ))

            # user sessions
            session.execute(sql_queries.insert_user_sessions, (
                r.user_id, r.session_id, r.item_in_session, r.first_name, r.last_name, r.artist, r.song
            ))
            # songs by user
            session.execute(sql_queries.insert_song_subscribers, (
                r.song, r.first_name, r.last_name
            ))
    print(f"loaded {line_count} lines ...")
    
print("method load_csv_data() created")

method load_csv_data() created


In [8]:
def drop_tables(session: Session):
    """
    drop all tables in session (keyspace)  
    
    :param 
        session: a cassandra.cluster.Session    
    """
    print("dropping database first...")
    from cassandra.cluster import Cluster
    try:
        session.execute(sql_queries.drop_songplay_sessions)
        session.execute(sql_queries.drop_user_sessions)
        session.execute(sql_queries.drop_song_subscribers)
        return session

    except Exception as e:
        print(e)
        raise e
        
print("method drop_tables() created")

method drop_tables() created


In [9]:
def database_init(session: Session):
    """
    create all tables in Cassandra 
    
    :param 
        session: a cassandra.cluster.Session    
    """

    print("initializing database...")
    from cassandra.cluster import Cluster
    try:

        drop_tables(session=session)

        # songplay sessions
        session.execute(sql_queries.create_songplay_sessions)

        # user_sessions
        session.execute(sql_queries.create_user_sessions)

        # song_subscribers
        session.execute(sql_queries.create_song_subscribers)
        return session

    except Exception as e:
        print(e)
        raise e
        
print("method database_init() created")

method database_init() created


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [10]:
# gather all files from event_data
csv_files = get_files(event_data="event_data")

# get data from all csv files into a single csv
master_file = merge_csv_data(file_path_list=csv_files)


Getting csv files from /home/workspace
dumped 1236 records
kept 6820 records
wrote 6821 lines to event_datafile_new.csv


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Create a Cluster, Keyspace and Schema

In [11]:
# create database cluster/schema and return session
database_init(session=cassandra_session)

initializing database...
dropping database first...


In [12]:
# load data into cassandra from the master file
load_csv_data(master_file, cassandra_session)

loading database...
loaded 6820 lines ...


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data


In [13]:
def query_songplays(session: Session, session_id : int, ordinal: int ):
    print(f"\n---- query songplays with sessionId={session_id}, and item={ordinal} ----")
    rows = session.execute(sql_queries.select_songplay_sessions, (session_id, ordinal))
    for row in rows:
        print(row)

def query_user_sessions(session: Session, user_id : int, session_id : int):
    print(f"\n---- query user sessions with userId={user_id} and sessionId={session_id} ----")
    rows = session.execute(sql_queries.select_user_sessions, (user_id, session_id))
    for row in rows:
        print(row)

def query_song_subscribers(session: Session, song: str):
    print(f"\n---- query all users listening to '{song} ----")
    rows = session.execute(sql_queries.select_song_subscribers, (song,))
    for row in rows:
        print(row)

print("created queries methods...")

created queries methods...


#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
print("\n---- Count table rows ---")

rows = cassandra_session.execute("select count(*) from songplay_sessions")
for row in rows:
    print(f"songplay_sessions has {row.count} rows")

rows = cassandra_session.execute("select count(*) from user_sessions")
for row in rows:
    print(f"user_sessions has {row.count} rows")

rows = cassandra_session.execute("select count(*) from song_subscribers")
for row in rows:
    print(f"song_subscribers has {row.count} rows")




---- Count table rows ---
songplay_sessions has 6820 rows
user_sessions has 6820 rows
song_subscribers has 6618 rows


#### Give me the artist, song title and song’s length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [15]:
query_songplays(session=cassandra_session, session_id=338, ordinal=4)


---- query songplays with sessionId=338, and item=4 ----
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', song_length=495.3073)


### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [16]:
query_user_sessions(session=cassandra_session, user_id=10, session_id=182)


---- query user sessions with userId=10 and sessionId=182 ----
Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession=0, firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession=1, firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession=2, firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3, firstname='Sylvie', lastname='Cruz')


#### Give me every user name (first and last) in my music app history who listened to the song ‘All Hands Against His Own’

In [17]:
query_song_subscribers(session=cassandra_session, song="All Hands Against His Own")


---- query all users listening to 'All Hands Against His Own ----
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Sara', lastname='Johnson')
Row(firstname='Tegan', lastname='Levine')


### Drop the tables before closing out the sessions

In [18]:
## TO-DO: Drop the table before closing out the sessions
drop_tables(session=cassandra_session)

dropping database first...


### Close the session and cluster connection¶

In [19]:
cassandra_session.execute("drop keyspace if exists event_data")
cassandra_session.shutdown()
cluster.shutdown()